In [73]:
# default_exp engine

# longtail

> API details.

In [74]:
#hide
from nbdev.showdoc import *

In [8]:
#export
import functools
import time
from collections import namedtuple
import csv
import pandas as pd
from types import MethodType
from pathlib import Path

In [4]:
#hide
def waste_some_time(num_times: int):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

In [2]:
#hide
!nbdev_build_lib

zsh:1: command not found: nbdev_build_lib


In [6]:
#export


class BlockTracker(object):
    
    def register(self, func):
        """Print the function signature and return value"""
        @functools.wraps(func)
        def wrapper_debug(*args, **kwargs):

            if "__kwdefaults__" in dir(func):
                d = func.__kwdefaults__
                if d is not None and "context" in d and d["context"] is None:
                    d["context"] = self

            args_repr = [repr(a) for a in args]                      # 1
            kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
            signature = ", ".join(args_repr + kwargs_repr)           # 3
            start_time = time.perf_counter()    # 1
            value = str(func(*args, **kwargs))
            end_time = time.perf_counter()      # 2
            run_time = end_time - start_time    # 3
            self.command_list.append(self.BlockCommand(name=func.__name__,
                                                       signature=signature,
                                                       value=value,
                                                       start=round(start_time, 2),
                                                       total=round(run_time, 9),
                                                       batch=self.batch,
                                                       stanza=self.stanza
                                                       ))

            if "__kwdefaults__" in dir(func):
                d = func.__kwdefaults__
                if d is not None and "context" in d and d["context"] == self:
                    d["context"] = None

            return value
        return wrapper_debug

    def __init__(self, file_name, stanza):
        self.path = file_name
        self.command_list = []
        self.code_list = []
        self.stanza = stanza
        self.start_time =  time.perf_counter()
        self.batch = str(self.start_time)
        self.BlockCommand = namedtuple("BlockCommand", "name signature value start total batch stanza")

    def data(self):
        return None
        #return pd.read_csv(self.path)

    def __enter__(self):
        self.start_time = time.perf_counter()
        return self.register

    def __exit__(self, type, value, traceback):
        self.start_time = time.perf_counter()

        for x in self.code_list:
            x.name(x.signature)

        with Path(self.path).open('a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(self.BlockCommand._fields)
            writer.writerows(self.command_list)

In [ ]:
#hide
with BlockTracker("./block.txt",stanza="Mid file 2") as f:
    f(print)("hello")
    f(waste_some_time)(300)
    f(print)("world")
    f(waste_some_time)(100)

In [1]:
#export
TrackerReg = [print]
class Tracker(object):

    def register(self, func):
        """Print the function signature and return value"""
        @functools.wraps(func)
        def wrapper_debug(self,*args, **kwargs):

            if "__kwdefaults__" in dir(func):
                d = func.__kwdefaults__
                if d is not None and "context" in d and d["context"] is None:
                    d["context"] = self

            args_repr = [repr(a) for a in args]                      # 1
            kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
            signature = ", ".join(args_repr + kwargs_repr)           # 3
            start_time = time.perf_counter()    # 1
            value = str(func(*args, **kwargs))
            end_time = time.perf_counter()      # 2
            run_time = end_time - start_time    # 3
            self.command_list.append(self.BlockCommand(name=func.__name__,
                                                       signature=signature,
                                                       value=value,
                                                       start=round(start_time, 2),
                                                       total=round(run_time, 9),
                                                       batch=self.batch,
                                                       stanza=self.stanza
                                                       ))

            if "__kwdefaults__" in dir(func):
                d = func.__kwdefaults__
                if d is not None and "context" in d and d["context"] == self:
                    d["context"] = None
            return value
        return wrapper_debug

    def __init__(self, file_name, stanza):
        self.path = file_name
        self.command_list = []
        self.code_list = []
        self.stanza = stanza
        self.start_time =  time.perf_counter()
        self.batch = str(self.start_time)
        self.BlockCommand = namedtuple("BlockCommand", "name signature value start total batch stanza")
        for x in TrackerReg:
            self.__setattr__(x.__name__,MethodType(self.register(x),self))

    def data(self):
        return pd.read_csv(self.path)

    def __enter__(self):
        self.start_time = time.perf_counter()
        return self

    def __exit__(self, type, value, traceback):
        self.start_time = time.perf_counter()

        for x in self.code_list:
            x.name(x.signature)

        with Path(self.path).open('a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(self.BlockCommand._fields)
            writer.writerows(self.command_list)



In [31]:
#hide
def myPrint(x):
    print(f"this words {x}")

TrackerReg.append(myPrint)

t = Tracker("./data/block.txt",stanza="Mid file 2")
t.myPrint("baloon")
8

this words baloon
hi


8

In [41]:
#hide
lastWrapper = ""

def capture(*args, context:Tracker = None):

    global lastWrapper
    if not context is None:
        lastWrapper = context
    for x in args:
        print(x)

TrackerReg.append(waste_some_time)
TrackerReg.append(capture)

In [42]:
#hide
with Tracker("./data/block.txt",stanza="Mid file 2") as f:
    f.capture("hello","world")
    f.print("human fish")
    f.waste_some_time(100)
lastWrapper

hello
world
human fish
